## What is 'shadow strength' ? 
Right now in the 'detailed sound propagation' I implement acoustic shadowing by calculating the effective reduction in sound pressure level only because of the object, and not because of spherical spreading. Eg. the bistatic target strength of the sphere I measured at 180$^{\circ}$ separation was -9 dB. This is including the spherical spreading of -6 dB from source to receiver. Therefore the effective 'shadow strenght' of the object is -3 dB. 


## Applying shadow strength to the multi target case:
If a single objects effective shadow strength is -3 dB - what is the effective shadow strength of multiple objects?
(see figure below) Is it correct to apply a 'lienar' calculation ie.

$Shadow \: strength_{Nbody} = \sum_{i=0}^{i=Nbody}shadow \: strength_{i}$

![](imgs/emitter-receiver.png)


Therefore while calculating the received level of sound with multiple bodies in between  -  it would be :
$  RL = SL -TL + Shadow \: strength_{Nbody}$

where $TL = 20log_{10}(R_{emitter-receiver}/R_{o})$ and $ SL$ is the source level at $R_{o}$.



### Vs doing it 'serially' : for example I could also calcualte the final received level through the bistatic sonar equation 21 in  [[1]](#references) :

$RL = SL - TL_{1} - TL{2} + TS $, where $TL_{1,2}$ are the transmission losses through spherical spreading $20log{10}(R_{1/2}/R_{o})$ and medium absorption-  which I will not consider right now. 

Let's treat the above case where $R_{emitter-receiver} = 3.0$ , SL = 100 dB SPL at 1m, TS = -9

With plain spherical spreading,a no obstacles, the received level will be :

In [70]:
import numpy as np 

In [177]:
SL = 100; TS= -9 ; R_emitterreceiver = 10.0
# Ro is 1m and so we won't have to do any more divisions. 
RL_noobstacles = SL - 20*np.log10(R_emitterreceiver)  
print(RL_noobstacles )

80.0


And now let's include the two obstacles. First let's treat the emitter-obstacle1-obstacle2 as the 3 body system. Obstacle 2 is treated as the receiver:

In [178]:
Ret1 = 1; Rt1t2 = 1; Rt2rec = R_emitterreceiver - Ret1 - Rt1t2
RL_obst2 = SL - 20*np.log10(Ret1) - 20*np.log10(Rt1t2) + TS
print(RL_obst2)

91.0


The received level is 99.5 dB SPL at Obstacle 2. We now will convert this received level into an 'effective' source level. We know sound arrived from 1.25 metres away, and thus the effective source level at 1metre is:


In [179]:
SL_obstacle2_1m =  RL_obst2 - 20*np.log10(1.0/(Ret1+Rt1t2) ) 
print(SL_obstacle2_1m)

97.02059991327963


Now, let's mimic what happens to the final received level of the sound at the real receiver using the same bistatic sonar equation:

In [180]:
RL_receiver  = SL_obstacle2_1m - 20*np.log10(Ret1+Rt1t2) - 20*np.log10(Rt2rec) + TS
print(RL_receiver)

63.938200260161125


The difference between the with and without obstacles received levels is thus:

In [76]:
RL_noobstacles - RL_receiver # dB

7.187849755184629

However, if what I expected was right I'd expected a difference of -6 dB while I got a difference   -- this tells me things are more complicated than expected  - which means I should write a script that calculates the received levels after shadowing iteratively. 

In [174]:
def calculate_RL_w_shadowing(distances, SL=100, TS=[-9]):
    '''Calculates received level of a call with acoustic shadowing included. 
    The received level of the call with shadowing is calculated with an iterative 
    application of the bistatic sonar equation. The TS used here is the bistatic 
    target strength at emitter-receiver angular separations of 180 degrees. 
    
    Parameters
    ----------
    distances : array-like, >= 3 entries. 
               The first entry is distance between emitter and closest target, followed by distance between 1st target
               and second target etc. The sum of all distances is the receiver-emitter distance

    SL : float.
         source level at 1metre distance in dB SPl re 20 muPa

    TS : array-like. 
         Bistatic target strengths of object at 180 degrees separation between emitter and receiver. 
         If multiple target strengths then a random choice is made within these values for each obstacle.
         This mimics each object having a different TS because of its orientation or position. 

    Returns
    -------
    RL_w_shadowing : float. 
                    Received level of sound after accounting for multiple object blocking the direct path. 
  
    '''
    all_distance_pairs = []
    for i, dist in enumerate(distances[:-1]):
        if i == 0:
            distance_pair = (dist,distances[i+1])
        else:
            distance_pair = ( np.sum(all_distance_pairs[-1]),distances[i+1])
        all_distance_pairs.append(distance_pair)
 
    

    SL_1m = SL
    for (r0,r1) in  all_distance_pairs:
        RL_apparent = SL_1m - 20*np.log10(r0) - 20*np.log10(r1) + np.random.choice(TS,1) # received level at the target
        SL_1m = RL_apparent - 20*np.log10(1/(r0+r1)) # equivalent source level at 1metre
    return(RL_apparent)
        
        
    

In [175]:
%time k = [calculate_RL_w_shadowing(np.random.choice(np.arange(0.5,0.9,0.05),5)) for k in range(1000)];


Wall time: 234 ms


In [188]:
calculate_RL_w_shadowing([2.5,0.5])

array([89.06179974])

<a id='references'></a>
### References
1. Cox H (1989) Fundamentals of Bistatic Sonar. Underwater Acoustic Data Processing, ed Chan YT (KLUWER ACADEMIC PUBLISHERS), pp 3–24.
